Preparing the data

In [101]:
import numpy as np
import pandas as pd
data = pd.read_csv("data425288.csv")

T = 25
N = data.ID.max()

y = np.log(data.Sales.values.reshape((N,T)))

X = np.ones((T,2))
X[:,1] = np.log(data.Price[:T])

In [236]:
#test params
K = 3

theta = np.random.rand(K,2) 
pi = np.ones(K)/K

    
likelihood = LogL(theta, pi, y, X)
display(likelihood.shape)
display(likelihood)

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


()

-inf

Log-likelihood evaluation

In [215]:
from scipy.stats import norm

def LogL(theta, pi, y, X):
    mu = np.dot(X,theta.T)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,3)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,3)
    
    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,3)
    
    #prod^T probs
    segments = np.prod(probs, axis=1) #(500,3)
    
    #sum^K pi_c prod^T probs
    combined = np.dot(segments, pi) #(500,)
    
    #sum^N log sum^K pi_c prod^T probs
    LogL = np.log(combined).sum() #(1,)
    return LogL

Expectation step

In [ ]:
def EStep(theta,pi,y,X):
    

Maximization step

In [ ]:
def MStep(W,y,X):

EM algorithm

In [ ]:
def EM(K,y,X):

Estimation implementation

In [ ]:
def Estimate(K, X=X, y=y, seed=1234):